In [1]:
import pandas as pd

In [2]:
full_df = pd.read_csv('dataset/train.csv')
df = full_df.copy()

In [3]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.2)

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

def update_train_df(dataframe):
    imputers = {
        'name_imputer': SimpleImputer(strategy='constant', fill_value='Unknown Unknown'),
        'num_imputer': SimpleImputer(strategy='median'),
        'cat_imputer': SimpleImputer(strategy='constant', fill_value='Unknown'),
    }

    num_col = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    cat_cols = ['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP']
    
    df = dataframe.copy()

    # Impute Part
    df.loc[:, 'Name'] = imputers['name_imputer'].fit_transform(df.loc[:, 'Name'].to_frame())
    df.loc[:, num_col] = imputers['num_imputer'].fit_transform(df.loc[:, num_col])
    df.loc[:, cat_cols] = imputers['cat_imputer'].fit_transform(df.loc[:, cat_cols])

    # Feature engineering Part
    df.loc[:, 'FirstName'] = df.loc[:, 'Name'].apply(lambda x: x.split()[0])
    df.loc[:, 'LastName'] = df.loc[:, 'Name'].apply(lambda x: x.split()[1])

    df.loc[:, 'PassengerGGGG'] = df.loc[:, 'PassengerId'].apply(lambda x: int(x.split('_')[0]))
    df.loc[:, 'PassengerPP'] = df.loc[:, 'PassengerId'].apply(lambda x: int(x.split('_')[1]))
    
    df.loc[:, 'NumberOfFellows'] = df.groupby('PassengerGGGG')['PassengerGGGG'].transform('count')
    df.loc[:, 'IsAlone'] = df.loc[:, 'NumberOfFellows'].apply(lambda x: 1 if x == 1 else 0)

    df['cabin_deck'] = df['Cabin'].apply(lambda x: x.split('/')[0] if x != 'Unknown' else 'U')
    df['cabin_num'] = df['Cabin'].apply(lambda x: int(x.split('/')[1]) if x != 'Unknown' else -1)
    df['cabin_side'] = df['Cabin'].apply(lambda x: x.split('/')[2] if x != 'Unknown' else 'U')

    df.loc[:, 'TotalSpend'] = df.loc[:, ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1)
    df['IsSpendingZero'] = (df['TotalSpend'] == 0).astype(int)

    # Removing unnecessary columns
    df.drop(['Cabin', 'Name', 'PassengerId'], inplace=True, axis=1)
    df.drop(['FirstName', 'LastName', 'PassengerGGGG'], inplace=True, axis=1)


    # std_scalers = {}

    # num_cols = df.select_dtypes('float').columns

    # for num_col in num_cols:
    #     scaler = StandardScaler()
    #     df.loc[:, num_col] = scaler.fit_transform(df.loc[:, num_col])

    #     std_scalers = 

    # Encoding 
    cat_data = df.select_dtypes('object')
    cat_data.loc[:, ['CryoSleep', 'VIP']] = cat_data.loc[:, ['CryoSleep', 'VIP']].astype('str')
    
    one_hot_encoder = OneHotEncoder(drop='first', dtype=int)
    
    cat_data_encoded = one_hot_encoder.fit_transform(cat_data)
    encoded_feature_names = one_hot_encoder.get_feature_names_out(cat_data.columns)
    
    cat_encoded = pd.DataFrame(cat_data_encoded.toarray(), columns = encoded_feature_names, index=cat_data.index)
    
    non_cat = df.drop(cat_data.columns, axis=1)
    df = pd.concat([non_cat, cat_encoded], axis=1)
    
    df.loc[:, 'Transported'] = df.loc[:, 'Transported'].astype(int)
    
    return df, imputers, one_hot_encoder

In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

def update_test_df(dataframe, imputers, one_hot_encoder):
    num_col = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    cat_cols = ['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP']
    
    df = dataframe.copy()

    # Impute Part
    df.loc[:, 'Name'] = imputers['name_imputer'].transform(df.loc[:, 'Name'].to_frame())
    df.loc[:, num_col] = imputers['num_imputer'].transform(df.loc[:, num_col])
    df.loc[:, cat_cols] = imputers['cat_imputer'].transform(df.loc[:, cat_cols])

    # Feature engineering Part
    df.loc[:, 'FirstName'] = df.loc[:, 'Name'].apply(lambda x: x.split()[0])
    df.loc[:, 'LastName'] = df.loc[:, 'Name'].apply(lambda x: x.split()[1])

    df.loc[:, 'PassengerGGGG'] = df.loc[:, 'PassengerId'].apply(lambda x: int(x.split('_')[0]))
    df.loc[:, 'PassengerPP'] = df.loc[:, 'PassengerId'].apply(lambda x: int(x.split('_')[1]))
    
    df.loc[:, 'NumberOfFellows'] = df.groupby('PassengerGGGG')['PassengerGGGG'].transform('count')
    df.loc[:, 'IsAlone'] = df.loc[:, 'NumberOfFellows'].apply(lambda x: 1 if x == 1 else 0)

    df['cabin_deck'] = df['Cabin'].apply(lambda x: x.split('/')[0] if x != 'Unknown' else 'U')
    df['cabin_num'] = df['Cabin'].apply(lambda x: int(x.split('/')[1]) if x != 'Unknown' else -1)
    df['cabin_side'] = df['Cabin'].apply(lambda x: x.split('/')[2] if x != 'Unknown' else 'U')

    df.loc[:, 'TotalSpend'] = df.loc[:, ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1)
    df['IsSpendingZero'] = (df['TotalSpend'] == 0).astype(int)

    # Removing unnecessary columns
    df.drop(['Cabin', 'Name', 'PassengerId'], inplace=True, axis=1)
    df.drop(['FirstName', 'LastName', 'PassengerGGGG'], inplace=True, axis=1)

    # Encoding
    cat_data = df.select_dtypes('object')
    cat_data.loc[:, ['CryoSleep', 'VIP']] = cat_data.loc[:, ['CryoSleep', 'VIP']].astype('str')
        
    cat_data_encoded = one_hot_encoder.transform(cat_data)
    encoded_feature_names = one_hot_encoder.get_feature_names_out(cat_data.columns)
    
    cat_encoded = pd.DataFrame(cat_data_encoded.toarray(), columns = encoded_feature_names, index=cat_data.index)
    
    non_cat = df.drop(cat_data.columns, axis=1)
    df = pd.concat([non_cat, cat_encoded], axis=1)
    
    return df

In [6]:
df_train_updated, train_imputers, train_ohe = update_train_df(df_train) 
df_test_updated = update_test_df(df_test, train_imputers, train_ohe)

/tmp/ipykernel_174626/3056761461.py:68: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1 1 0 ... 0 1 1]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[:, 'Transported'] = df.loc[:, 'Transported'].astype(int)


In [7]:
X_train = df_train_updated.drop('Transported', axis=1)
y_train = df_train_updated.loc[:, 'Transported']

X_test = df_test_updated.drop('Transported', axis=1)
y_test = df_test_updated.loc[:, 'Transported']

In [8]:
from sklearn.ensemble import StackingClassifier, VotingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [9]:
lgbm_model = LGBMClassifier(
    num_leaves=121,
    max_depth=4,
    learning_rate=0.04344827588960371,
    n_estimators=408,
    subsample=0.8970512932941775,
    colsample_bytree=0.8831320655300392,
    reg_alpha=0.028950177705629465,
    reg_lambda=0.12806231299056023,
    min_child_samples=30,
    random_state=42
)

xgb_model = XGBClassifier(
    n_estimators=817,
    max_depth=3,
    learning_rate=0.0816525631511912,
    subsample=0.9618214514489957,
    colsample_bytree=0.6390303515291257,
    gamma=0.2256057696902123,
    reg_alpha=0.40266566984067254,
    reg_lambda=0.14568814833206822,
    min_child_weight=3,
    eval_metric='logloss',
    random_state=42
)

rf_model = RandomForestClassifier(
    n_estimators=758,
    max_depth=20,
    min_samples_split=19,
    min_samples_leaf=20,
    max_features=None,
    bootstrap=True,
    criterion='entropy',
    random_state=42
)

svc_model = SVC(
    C=7.343347632061625,
    kernel='poly',
    degree=2,
    gamma='auto',
    coef0=0.7604473808714275,
    probability=True,
    random_state=42
)

logreg_model = LogisticRegression(
    C=0.04365892318987059,
    penalty='l1',
    solver='liblinear',
    random_state=42
)

knn_model = KNeighborsClassifier(
    n_neighbors=40,
    weights='distance',
    p=1,
    metric='minkowski'
)

In [10]:
voting_clf = VotingClassifier(
    estimators=[
        ('lgbm', lgbm_model),
        ('xgb', xgb_model),
        ('rf', rf_model),
        ('svc', make_pipeline(StandardScaler(), svc_model)),
        ('knn', make_pipeline(StandardScaler(), knn_model)),
        ('logreg', make_pipeline(StandardScaler(), logreg_model))
    ],
    voting='soft'
)

voting_clf.fit(X_train, y_train)
y_pred_voting = voting_clf.predict(X_test)

print("Voting Classifier Accuracy:", accuracy_score(y_test, y_pred_voting))

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 3485, number of negative: 3469
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000397 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1922
[LightGBM] [Info] Number of data points in the train set: 6954, number of used features: 31
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.501150 -> initscore=0.004602
[LightGBM] [Info] Start training from score 0.004602
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

In [11]:
train, train_imputers, train_ohe = update_train_df(full_df)

/tmp/ipykernel_174626/3056761461.py:68: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0 1 0 ... 1 0 1]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[:, 'Transported'] = df.loc[:, 'Transported'].astype(int)


In [12]:
test_df = pd.read_csv('dataset/test.csv')
test = update_test_df(test_df, train_imputers, train_ohe)

In [13]:
X = train.drop('Transported', axis=1)
y = train.loc[:, 'Transported']

In [14]:
voting_clf.fit(X, y)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 4378, number of negative: 4315
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000756 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1924
[LightGBM] [Info] Number of data points in the train set: 8693, number of used features: 31
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503624 -> initscore=0.014495
[LightGBM] [Info] Start training from score 0.014495
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

VotingClassifier(estimators=[('lgbm',
                              LGBMClassifier(colsample_bytree=0.8831320655300392,
                                             learning_rate=0.04344827588960371,
                                             max_depth=4, min_child_samples=30,
                                             n_estimators=408, num_leaves=121,
                                             random_state=42,
                                             reg_alpha=0.028950177705629465,
                                             reg_lambda=0.12806231299056023,
                                             subsample=0.8970512932941775)),
                             ('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            call...
                                                   random_state=42))])),
                             ('knn',
                              Pipeline(steps=[('standardscaler',
                                               StandardScaler()),
                                              ('kneighborsclassifier',
                                               KNeighborsClassifier(n_neighbors=40,
                                                                    p=1,
                                                                    weights='distance'))])),
                             ('logreg',
                              Pipeline(steps=[('standardscaler',
                                               StandardScaler()),
                                              ('logisticregression',
                                               LogisticRegression(C=0.04365892318987059,
                                                                  penalty='l1',
                                                                  random_state=42,
                                                                  solver='liblinear'))]))],
                 voting='soft')

In [15]:
y_pred = voting_clf.predict(test)
y_pre = y_pred.astype(bool)

In [16]:
test_df = pd.read_csv('dataset/test.csv')


submission = pd.DataFrame({
    'PassengerId': test_df['PassengerId'],
    'Transported' : y_pred.astype(bool)
})
submission

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,True


In [17]:
submission.to_csv('voting-soft-80.16.csv', index=False)

In [18]:
voting_clf = VotingClassifier(
    estimators=[
        ('lgbm', lgbm_model),
        ('xgb', xgb_model),
        ('rf', rf_model),
        ('svc', make_pipeline(StandardScaler(), svc_model)),
        ('knn', make_pipeline(StandardScaler(), knn_model)),
        ('logreg', make_pipeline(StandardScaler(), logreg_model))
    ],
    voting='hard'
)

voting_clf.fit(X_train, y_train)
y_pred_voting = voting_clf.predict(X_test)

print("Voting Classifier Accuracy:", accuracy_score(y_test, y_pred_voting))

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 3485, number of negative: 3469
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001499 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1922
[LightGBM] [Info] Number of data points in the train set: 6954, number of used features: 31
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.501150 -> initscore=0.004602
[LightGBM] [Info] Start training from score 0.004602
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

In [19]:
train, train_imputers, train_ohe = update_train_df(full_df)

/tmp/ipykernel_174626/3056761461.py:68: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0 1 0 ... 1 0 1]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[:, 'Transported'] = df.loc[:, 'Transported'].astype(int)


In [20]:
test_df = pd.read_csv('dataset/test.csv')
test = update_test_df(test_df, train_imputers, train_ohe)

In [21]:
X = train.drop('Transported', axis=1)
y = train.loc[:, 'Transported']

In [22]:
voting_clf.fit(X, y)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 4378, number of negative: 4315
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000789 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1924
[LightGBM] [Info] Number of data points in the train set: 8693, number of used features: 31
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503624 -> initscore=0.014495
[LightGBM] [Info] Start training from score 0.014495
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

VotingClassifier(estimators=[('lgbm',
                              LGBMClassifier(colsample_bytree=0.8831320655300392,
                                             learning_rate=0.04344827588960371,
                                             max_depth=4, min_child_samples=30,
                                             n_estimators=408, num_leaves=121,
                                             random_state=42,
                                             reg_alpha=0.028950177705629465,
                                             reg_lambda=0.12806231299056023,
                                             subsample=0.8970512932941775)),
                             ('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            call...
                                                   probability=True,
                                                   random_state=42))])),
                             ('knn',
                              Pipeline(steps=[('standardscaler',
                                               StandardScaler()),
                                              ('kneighborsclassifier',
                                               KNeighborsClassifier(n_neighbors=40,
                                                                    p=1,
                                                                    weights='distance'))])),
                             ('logreg',
                              Pipeline(steps=[('standardscaler',
                                               StandardScaler()),
                                              ('logisticregression',
                                               LogisticRegression(C=0.04365892318987059,
                                                                  penalty='l1',
                                                                  random_state=42,
                                                                  solver='liblinear'))]))])

In [23]:
y_pred = voting_clf.predict(test)
y_pre = y_pred.astype(bool)

In [24]:
test_df = pd.read_csv('dataset/test.csv')


submission = pd.DataFrame({
    'PassengerId': test_df['PassengerId'],
    'Transported' : y_pred.astype(bool)
})
submission

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,True


In [25]:
submission.to_csv('voting-hard-80.csv', index=False)

In [26]:
stacking_clf = StackingClassifier(
    estimators=[
        ('lgbm', lgbm_model),
        ('xgb', xgb_model),
        ('rf', rf_model),
        ('svc', make_pipeline(StandardScaler(), svc_model)),
        ('knn', make_pipeline(StandardScaler(), knn_model)),
        ('logreg', make_pipeline(StandardScaler(), logreg_model))
    ],
    final_estimator=make_pipeline(StandardScaler(), LogisticRegression(random_state=42, max_iter=1000)),
    passthrough=True
)

stacking_clf.fit(X_train, y_train)
y_pred_stacking = stacking_clf.predict(X_test)

print("Stacking Classifier Accuracy:", accuracy_score(y_test, y_pred_stacking))

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 3485, number of negative: 3469
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000466 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1922
[LightGBM] [Info] Number of data points in the train set: 6954, number of used features: 31
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.501150 -> initscore=0.004602
[LightGBM] [Info] Start training from score 0.004602
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

In [27]:
train, train_imputers, train_ohe = update_train_df(full_df)

/tmp/ipykernel_174626/3056761461.py:68: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0 1 0 ... 1 0 1]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[:, 'Transported'] = df.loc[:, 'Transported'].astype(int)


In [28]:
test_df = pd.read_csv('dataset/test.csv')
test = update_test_df(test_df, train_imputers, train_ohe)

In [29]:
X = train.drop('Transported', axis=1)
y = train.loc[:, 'Transported']

In [30]:
stacking_clf.fit(X, y)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 4378, number of negative: 4315
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000631 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1924
[LightGBM] [Info] Number of data points in the train set: 8693, number of used features: 31
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503624 -> initscore=0.014495
[LightGBM] [Info] Start training from score 0.014495
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

StackingClassifier(estimators=[('lgbm',
                                LGBMClassifier(colsample_bytree=0.8831320655300392,
                                               learning_rate=0.04344827588960371,
                                               max_depth=4,
                                               min_child_samples=30,
                                               n_estimators=408, num_leaves=121,
                                               random_state=42,
                                               reg_alpha=0.028950177705629465,
                                               reg_lambda=0.12806231299056023,
                                               subsample=0.8970512932941775)),
                               ('xgb',
                                XGBClassifier(base_score=None, booster=None,
                                              ca...
                                                                      weights='distance'))])),
                               ('logreg',
                                Pipeline(steps=[('standardscaler',
                                                 StandardScaler()),
                                                ('logisticregression',
                                                 LogisticRegression(C=0.04365892318987059,
                                                                    penalty='l1',
                                                                    random_state=42,
                                                                    solver='liblinear'))]))],
                   final_estimator=Pipeline(steps=[('standardscaler',
                                                    StandardScaler()),
                                                   ('logisticregression',
                                                    LogisticRegression(max_iter=1000,
                                                                       random_state=42))]),
                   passthrough=True)

In [31]:
y_pred = voting_clf.predict(test)
y_pre = y_pred.astype(bool)

In [32]:
test_df = pd.read_csv('dataset/test.csv')


submission = pd.DataFrame({
    'PassengerId': test_df['PassengerId'],
    'Transported' : y_pred.astype(bool)
})
submission

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,True


In [33]:
submission.to_csv('stacking-logrg-80.27.csv', index=False)